## Q1K Questionnaires tracking sheet

In [65]:
# Import modules
import os
import pandas as pd
from pathlib import Path
import numpy as np
import glob
import shutil
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio

import docx
from docx.enum.section import WD_ORIENT
from docx.enum.section import WD_SECTION
from docx.enum.text import WD_ALIGN_PARAGRAPH
pio.renderers.default = "plotly_mimetype+notebook"
from docx.shared import Pt
import seaborn as sns
import kaleido
#import openpyxl

In [66]:
# Function to check if file name contains task information
def has_task_info(file_name, task_info):
    return task_info in file_name


# 1. Extracting REDCap questionnaire

In [67]:
# Select the date
date="2025_05_07"

In [69]:
for file in glob.glob(f"../source/{date}/questionnaires/*"):
    if "Quest" in file:
       # quest_df = pd.read_csv(file)
        print( "Questionnaire file is: " , file)
    if "LABEL" in file:
        quest_df = pd.read_csv(file)
        print( "Labels file is: " , file)

Labels file is:  ../source/2025_05_07/questionnaires\DATA_LABELS.csv
Questionnaire file is:  ../source/2025_05_07/questionnaires\Q1KDatabase-QuestionnairesTracki_DATA_2025-05-07_1526.csv


#### Check to see the names of the columns

In [70]:
quest_df.columns

Index(['Record ID', 'Event Name', 'Proband ID', 'Is this the proband?',
       'Relative ID (generated)', 'Participant's status:', 'Age At Enrollment',
       '2.\tQuestionnaire data', 'Questionnaires sent:', 'Complete?',
       'Questionnaires complete?', 'Complete?.1',
       'Date of questionnaire withdrawal', 'Complete?.2', 'Complete?.3',
       'Complete?.4', 'Complete?.5'],
      dtype='object')

### Change ambiguous column names


In [71]:

column_renames = {"Complete?.1": "Phase_3_sent", 
                  "Complete?.2": "SRS_4_", 
                    "Complete?.3": "SRS_5_",
                    "Complete?.4": "SRS_6_",
                    "Complete?.5": "SRS_7_", 
                    # Add here more column renames as needed
                    }

# Rename columns in the DataFrame
quest_df.rename(columns=column_renames, inplace=True)

# Drop columns that are not needed
quest_df.drop(columns=["Event Name"], inplace=True)

### Merge all rows with the same ID to have one row per participant


In [72]:
quest_df = quest_df.groupby('Record ID', as_index=False).first()  

### Check that column names were actually updated

In [73]:
quest_df.columns

Index(['Record ID', 'Proband ID', 'Is this the proband?',
       'Relative ID (generated)', 'Participant's status:', 'Age At Enrollment',
       '2.\tQuestionnaire data', 'Questionnaires sent:', 'Complete?',
       'Questionnaires complete?', 'Phase_3_sent',
       'Date of questionnaire withdrawal', 'SRS_4_', 'SRS_5_', 'SRS_6_',
       'SRS_7_'],
      dtype='object')

## 2. Merge all SRS columns into one

In [74]:
# Merge SRS Columns 
srs_colunms = [col for col in quest_df.columns if "SRS" in col]
srs_colunms

['SRS_4_', 'SRS_5_', 'SRS_6_', 'SRS_7_']

In [75]:
# Create a new column 'SRS_Complete' based on the SRS columns
quest_df['SRS_Complete'] = quest_df[srs_colunms].apply(
    lambda row: "Complete" if (row == "Complete").any() else "Incomplete", axis=1
)

In [76]:
quest_df

,Record ID,Proband ID,Is this the proband?,Relative ID (generated),Participant's status:,Age At Enrollment,2.\tQuestionnaire data,Questionnaires sent:,Complete?,Questionnaires complete?,Phase_3_sent,Date of questionnaire withdrawal,SRS_4_,SRS_5_,SRS_6_,SRS_7_,SRS_Complete
0,100,Q1K_HSJ_100100_P,Yes,None,None,6.499905,Yes,2024-02-02,Complete,Yes,Complete,None,Incomplete,Complete,Incomplete,Incomplete,Complete
1,101,None,No,Q1K_HSJ_100100_S1,Relative affected (genetic carrier or ASD/NDD),3.293816,Yes,2024-02-05,Complete,Yes,Complete,None,Complete,Incomplete,Incomplete,Incomplete,Complete
2,102,None,No,Q1K_HSJ_100100_M1,Relative non-affected,26.869819,Yes,2024-02-05,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete
3,103,None,No,Q1K_HSJ_100100_F1,Relative affected (genetic carrier or ASD/NDD),30.059481,Yes,2024-02-05,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete
4,104,Q1K_HSJ_100104_P,Yes,None,None,NaN,Yes,2024-02-14,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,94,None,No,Q1K_HSJ_10093_M1,Relative affected (genetic carrier or ASD/NDD),27.970458,Yes,2024-02-24,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete
749,95,None,No,Q1K_HSJ_10093_O1,Relative non-affected,56.899183,Yes,2024-02-24,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete
750,97,Q1K_HSJ_10097_P,Yes,None,None,7.964571,Yes,2024-02-05,Complete,Yes,Complete,None,Incomplete,Complete,Incomplete,Incomplete,Complete
751,98,None,No,Q1K_HSJ_10097_M1,Relative non-affected,37.835137,Yes,2024-02-05,Complete,Yes,Complete,None,Incomplete,Incomplete,Complete,Incomplete,Complete


In [77]:
quest_df.columns

Index(['Record ID', 'Proband ID', 'Is this the proband?',
       'Relative ID (generated)', 'Participant's status:', 'Age At Enrollment',
       '2.\tQuestionnaire data', 'Questionnaires sent:', 'Complete?',
       'Questionnaires complete?', 'Phase_3_sent',
       'Date of questionnaire withdrawal', 'SRS_4_', 'SRS_5_', 'SRS_6_',
       'SRS_7_', 'SRS_Complete'],
      dtype='object')

### 3. Create participant coluns to see where there are bottlenecks in the processing

In [78]:
steps = ["Participant's status:", 'Age At Enrollment',
       "2.\tQuestionnaire data", 'Questionnaires sent:', 'Complete?',
       'Questionnaires complete?', 'Phase_3_sent']


# Bootleneck
quest_df['First_missing'] = quest_df[steps].apply(
    lambda row: next((step for step in steps if pd.isna(row[step])), 'none'),
    axis=1
)


# 2. Flag where one step is NA and next is not NA
def flag_discrepancy(row):
    for i in range(len(steps) - 1):
        if pd.isna(row[steps[i]]) and not pd.isna(row[steps[i + 1]]):
            return True
    return False

quest_df['discrepancy'] = quest_df.apply(flag_discrepancy, axis=1)

# Change discrepancy from True to Yes and False to No
quest_df['discrepancy'] = quest_df['discrepancy'].replace({True: 'Yes', False: 'No'})

# 3. Count of NAs
quest_df['Number Missing'] = quest_df[steps].isna().sum(axis=1)

# 4. List of steps with NAs
quest_df['Missing variables'] = quest_df[steps].apply(
    lambda row: [step for step in steps if pd.isna(row[step])],
    axis=1
)

# Add columns for manual check
quest_df['Manual check'] = np.nan

# Add columns for further details
quest_df['Details'] = np.nan

# Add columns for fixed
quest_df['Fixed'] = np.nan

In [79]:
quest_df

,Record ID,Proband ID,Is this the proband?,Relative ID (generated),Participant's status:,Age At Enrollment,2.\tQuestionnaire data,Questionnaires sent:,Complete?,Questionnaires complete?,...,SRS_6_,SRS_7_,SRS_Complete,First_missing,discrepancy,Number Missing,Missing variables,Manual check,Details,Fixed
0,100,Q1K_HSJ_100100_P,Yes,None,None,6.499905,Yes,2024-02-02,Complete,Yes,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
1,101,None,No,Q1K_HSJ_100100_S1,Relative affected (genetic carrier or ASD/NDD),3.293816,Yes,2024-02-05,Complete,Yes,...,Incomplete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
2,102,None,No,Q1K_HSJ_100100_M1,Relative non-affected,26.869819,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
3,103,None,No,Q1K_HSJ_100100_F1,Relative affected (genetic carrier or ASD/NDD),30.059481,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
4,104,Q1K_HSJ_100104_P,Yes,None,None,NaN,Yes,2024-02-14,Complete,Yes,...,Complete,Incomplete,Complete,Participant's status:,Yes,2,"[Participant's status:, Age At Enrollment]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,94,None,No,Q1K_HSJ_10093_M1,Relative affected (genetic carrier or ASD/NDD),27.970458,Yes,2024-02-24,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
749,95,None,No,Q1K_HSJ_10093_O1,Relative non-affected,56.899183,Yes,2024-02-24,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
750,97,Q1K_HSJ_10097_P,Yes,None,None,7.964571,Yes,2024-02-05,Complete,Yes,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
751,98,None,No,Q1K_HSJ_10097_M1,Relative non-affected,37.835137,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN


In [80]:
# Save the DataFrame to a CSV file

# Make sure output directory exists
Path(f"../output/{date}/questionnaires/").mkdir(parents=True, exist_ok=True)


quest_df.to_csv(f"../output/{date}/questionnaires/questionnaires_completed_{date}.csv", index=False)



In [81]:
quest_df

,Record ID,Proband ID,Is this the proband?,Relative ID (generated),Participant's status:,Age At Enrollment,2.\tQuestionnaire data,Questionnaires sent:,Complete?,Questionnaires complete?,...,SRS_6_,SRS_7_,SRS_Complete,First_missing,discrepancy,Number Missing,Missing variables,Manual check,Details,Fixed
0,100,Q1K_HSJ_100100_P,Yes,None,None,6.499905,Yes,2024-02-02,Complete,Yes,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
1,101,None,No,Q1K_HSJ_100100_S1,Relative affected (genetic carrier or ASD/NDD),3.293816,Yes,2024-02-05,Complete,Yes,...,Incomplete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
2,102,None,No,Q1K_HSJ_100100_M1,Relative non-affected,26.869819,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
3,103,None,No,Q1K_HSJ_100100_F1,Relative affected (genetic carrier or ASD/NDD),30.059481,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
4,104,Q1K_HSJ_100104_P,Yes,None,None,NaN,Yes,2024-02-14,Complete,Yes,...,Complete,Incomplete,Complete,Participant's status:,Yes,2,"[Participant's status:, Age At Enrollment]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,94,None,No,Q1K_HSJ_10093_M1,Relative affected (genetic carrier or ASD/NDD),27.970458,Yes,2024-02-24,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
749,95,None,No,Q1K_HSJ_10093_O1,Relative non-affected,56.899183,Yes,2024-02-24,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
750,97,Q1K_HSJ_10097_P,Yes,None,None,7.964571,Yes,2024-02-05,Complete,Yes,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
751,98,None,No,Q1K_HSJ_10097_M1,Relative non-affected,37.835137,Yes,2024-02-05,Complete,Yes,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN


### 4. Make graphs with how many participants completed each SRS column

In [88]:
# Plot how many have compelted each of the SRS questionnaires

# Create a new DataFrame for the SRS columns
srs_df = quest_df[['SRS_Complete']].copy()

# count the number of occurrences of each value in the SRS_Complete column
srs_counts = srs_df.value_counts().reset_index()
srs_counts.columns = ['SRS_Complete', 'Count']
# Create a bar plot
fig = px.bar(srs_counts, x='SRS_Complete', y='Count', title='SRS Completion Status',
             color='SRS_Complete', text='Count')
# Update the layout
fig.update_layout(
    xaxis_title='SRS Completion Status',
    yaxis_title='Count',
    showlegend=False,
    title_x=0.5,  # Center the title
    title_y=0.95,  # Adjust the vertical position of the title
    font=dict(size=14)  # Set font size for all text in the figure
)
# Show the figure
fig.show()

# Save the figure as a PNG file
fig.write_image(f"../output/{date}/questionnaires/SRS_Completion_Status_{date}.png", width=800, height=600)

In [89]:
# Create a df for sankey plots
quest_df_sankey = quest_df.copy()

# For each column, covert to 0 if nan and 1 if not na

for col in steps:
    quest_df_sankey[col] = quest_df_sankey[col].apply(lambda x: 0 if pd.isna(x) else 1)

In [90]:
quest_df_sankey

,Record ID,Proband ID,Is this the proband?,Relative ID (generated),Participant's status:,Age At Enrollment,2.\tQuestionnaire data,Questionnaires sent:,Complete?,Questionnaires complete?,...,SRS_6_,SRS_7_,SRS_Complete,First_missing,discrepancy,Number Missing,Missing variables,Manual check,Details,Fixed
0,100,Q1K_HSJ_100100_P,Yes,None,0,1,1,1,1,1,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
1,101,None,No,Q1K_HSJ_100100_S1,1,1,1,1,1,1,...,Incomplete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
2,102,None,No,Q1K_HSJ_100100_M1,1,1,1,1,1,1,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
3,103,None,No,Q1K_HSJ_100100_F1,1,1,1,1,1,1,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
4,104,Q1K_HSJ_100104_P,Yes,None,0,0,1,1,1,1,...,Complete,Incomplete,Complete,Participant's status:,Yes,2,"[Participant's status:, Age At Enrollment]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,94,None,No,Q1K_HSJ_10093_M1,1,1,1,1,1,1,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
749,95,None,No,Q1K_HSJ_10093_O1,1,1,1,1,1,1,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN
750,97,Q1K_HSJ_10097_P,Yes,None,0,1,1,1,1,1,...,Incomplete,Incomplete,Complete,Participant's status:,Yes,1,[Participant's status:],NaN,NaN,NaN
751,98,None,No,Q1K_HSJ_10097_M1,1,1,1,1,1,1,...,Complete,Incomplete,Complete,none,No,0,[],NaN,NaN,NaN


# 5. Create sankeu diagram of how many participants lost at each stage

In [91]:
# Create a sankey plot for the questionnaire data
# Define steps 
df = quest_df_sankey.copy()

# Calculate the number of subjects at each step
step_counts = {step: df[step].sum().astype(int) for step in steps}


# Create nodes and links for the Sankey diagram
nodes = steps 
node_indices = {node: i for i, node in enumerate(nodes)}

links = []

# Identify the first step dynamically
first_step = steps[0]

# Add a dummy link for the first step to represent its full size
first_step_total = step_counts[first_step]
first_step_to_next = df[(df[first_step] == 1) & (df[steps[1]] == 1)].shape[0]
dummy_value = first_step_total - first_step_to_next

# Calculate lost subjects at each step
lost_counts = {}
for i in range(len(steps) - 1):
    source = steps[i]
    target = steps[i + 1]
    value = df[(df[source] == 1) & (df[target] == 1)].shape[0]
    lost_counts[source] = step_counts[source] - value



# Add the dummy link but make it transparent
links.append({
    "source": node_indices[first_step],
    "target": node_indices[first_step],
    "value": dummy_value,
    "color": "rgba(0, 0, 0, 0)"  # Transparent color
})

# Add the actual flow from the first step to the next step
links.append({
    "source": node_indices[first_step],
    "target": node_indices[steps[1]],
    "value": first_step_to_next
})

# Calculate flows between other steps
for i in range(1, len(steps) - 1):  # Start from 1 to skip the first step
    source = steps[i]
    target = steps[i + 1]
    value = df[(df[source] == 1) & (df[target] == 1)].shape[0]
    links.append({
        "source": node_indices[source],
        "target": node_indices[target],
        "value": value
    })


# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=[f"{node}<br>({step_counts.get(node)})" for node in nodes],  # Add counts to labels
    ),
    link=dict(
        source=[link["source"] for link in links],
        target=[link["target"] for link in links],
        value=[link["value"] for link in links],
        color=[link.get("color", "rgba(170, 170, 170, 0.6)") for link in links]  # Default link color
    )
))

# Add text annotations at the bottom left
step_annotation = "<br>".join([f"{step}: {step_counts[step]} subjects" for step in steps])
#  task_annotation = "<br>".join([f"{task}: {task_counts[task]} subjects" for task in eeg_tasks])
lost_annotation = "<br>".join([f"{step}: {lost_counts.get(step, 0)} subjects lost" for step in steps[:-1]])


# Add step annotations to the bottom left
fig.add_annotation(
    x=0.05,  # Bottom left
    y=-0.32,  # Move further down for clarity
    text=f"<b>Stage:</b><br>{step_annotation}",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=dict(size=18),  # Reduce font slightly for better spacing
    align="left"
)

# Add lost subjects annotation below step 
fig.add_annotation(
    x=0.55,  # Align horizontally
    y=-0.32,  # Keep same y-level for uniform height
    text=f"<b>Lost Subjects:</b><br>{lost_annotation}<br>&nbsp;",  # Ensure consistent height
    showarrow=False,
    xref="paper",
    yref="paper",
    font=dict(size=18),
    align="left"
)


fig.update_layout(
            title=dict(
        text= f"Questionnaires Tracking Report {date}", 
        x=0.5,   
        xanchor="center",   
        font=dict(size=24)  
    ),
    margin=dict(t=50, b=250, l=100, r=100),  
            #   margin=dict(t=50, b=250),  # More bottom space to prevent overlap


    # Increase bottom margin to prevent overlap
    width=1500,
    height=800
)



html_output_dir= f"../output/{date}/questionnaires/figures/"
# Print current directory 
if not os.path.exists(html_output_dir):
    os.makedirs(html_output_dir)       

# Export to HTML
fig.write_html(html_output_dir+f"/sankey_plot_{date}.html")


In [92]:


# Save the figure as a PNG file
fig.write_image(html_output_dir + f"/questionnaires_tracking_report_{date}.png", format="png", width=1500, height=800)

